In [1]:
import os
import sys
import datetime

import pickle
import tensorflow as tf
from sklearn.model_selection import train_test_split

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))

from SRModels.deep_learning_models.ESRGAN_model import ESRGAN
from SRModels.loading_methods import load_dataset_as_patches
from SRModels.constants import ESRGAN_PATCH_SIZE, ESRGAN_STRIDE, RANDOM_SEED, ESRGAN_SCALE_FACTOR

c:\Users\bgmanuel\anaconda3\envs\py310\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\bgmanuel\anaconda3\envs\py310\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.

In [2]:
# Enable memory growth (prevents full pre-allocation)
for gpu in tf.config.experimental.list_physical_devices("GPU"):
    try:
        tf.config.experimental.set_memory_growth(gpu, True)
    except Exception as e:
        print(f"Memory growth not set: {e}")

In [3]:
HR_ROOT = os.path.abspath(os.path.join(os.getcwd(), "../../data/images/HR"))
LR_ROOT = os.path.abspath(os.path.join(os.getcwd(), "../../data/images/LR"))

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
directory = f"models/ESRGAN/ESRGAN_{timestamp}"
grid_figures_directory = f"{directory}/grid_figures"

In [ ]:
X, Y = load_dataset_as_patches(HR_ROOT, LR_ROOT, mode="scale", patch_size=ESRGAN_PATCH_SIZE, stride=ESRGAN_STRIDE, scale_factor=ESRGAN_SCALE_FACTOR)

# Reduce dataset size by percentage
X = X[:int(len(X) * 0.7)]
Y = Y[:int(len(Y) * 0.7)]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, shuffle=True, random_state=RANDOM_SEED)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, shuffle=True, random_state=RANDOM_SEED)

print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}")
print(f"X_val shape: {X_val.shape}, Y_val shape: {Y_val.shape}")
print(f"X_test shape: {X_test.shape}, Y_test shape: {Y_test.shape}")

X_train shape: (45761, 24, 24, 3), Y_train shape: (45761, 48, 48, 3)
X_val shape: (5085, 24, 24, 3), Y_val shape: (5085, 48, 48, 3)
X_test shape: (5650, 24, 24, 3), Y_test shape: (5650, 48, 48, 3)


In [5]:
BATCH_SIZE = 16     # Ajustar según memoria GPU

# Se define también el dataset de test (solo para evaluación posterior)
# Se normaliza a [-1,1] para que coincida con lo usado en entrenamiento
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(BATCH_SIZE)
test_dataset = test_dataset.map(lambda x,y: (x*2.0 - 1.0, y*2.0 - 1.0), num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)

In [6]:
model = ESRGAN()

model.setup_model(
    scale_factor=2, 
    growth_channels=8, 
    num_rrdb_blocks=4, 
    input_shape=X_train.shape[1:],
    output_shape=Y_train.shape[1:],
    from_trained=False
)

GENERATOR SUMMARY
Model: "Generator"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 lr_input (InputLayer)          [(None, 24, 24, 3)]  0           []                               
                                                                                                  
 initial_conv (Conv2D)          (None, 24, 24, 64)   1792        ['lr_input[0][0]']               
                                                                                                  
 rrdb_0_dense1_conv1 (Conv2D)   (None, 24, 24, 8)    4616        ['initial_conv[0][0]']           
                                                                                                  
 rrdb_0_dense1_concat1 (Concate  (None, 24, 24, 72)  0           ['initial_conv[0][0]',           
 nate)                                                            'rrdb_

In [7]:
# Train ESRGAN and capture callbacks for metrics
epoch_losses, time_cb, mem_cb = model.fit(
    X_train=X_train, 
    Y_train=Y_train, 
    X_val=X_val, 
    Y_val=Y_val,
    epochs=1, 
    batch_size=BATCH_SIZE,
    save_dir=grid_figures_directory,
)

Training on GPU: ['/physical_device:GPU:0']
Epoch 1/1
  Step 10/2861 G_loss=74.4660 D_loss=1.3864 PSNR=19.52 SSIM=0.3380
  Step 20/2861 G_loss=55.3448 D_loss=1.3859 PSNR=20.94 SSIM=0.5054
  Step 30/2861 G_loss=49.8604 D_loss=1.3859 PSNR=22.44 SSIM=0.6041
  Step 40/2861 G_loss=61.6402 D_loss=1.3859 PSNR=22.82 SSIM=0.6248
  Step 50/2861 G_loss=42.1344 D_loss=1.3859 PSNR=23.87 SSIM=0.6844
  Step 60/2861 G_loss=42.5883 D_loss=1.3857 PSNR=23.55 SSIM=0.7016
  Step 70/2861 G_loss=31.7325 D_loss=1.3858 PSNR=25.41 SSIM=0.7402
  Step 80/2861 G_loss=27.5734 D_loss=1.3858 PSNR=26.28 SSIM=0.7342
  Step 90/2861 G_loss=44.4819 D_loss=1.3854 PSNR=23.87 SSIM=0.7344
  Step 100/2861 G_loss=33.3793 D_loss=1.3855 PSNR=24.96 SSIM=0.7082
  Step 110/2861 G_loss=26.9877 D_loss=1.3851 PSNR=26.41 SSIM=0.7779
  Step 120/2861 G_loss=29.9423 D_loss=1.3845 PSNR=26.53 SSIM=0.7157
  Step 130/2861 G_loss=24.9139 D_loss=1.3851 PSNR=27.47 SSIM=0.8235
  Step 140/2861 G_loss=29.2245 D_loss=1.3834 PSNR=26.60 SSIM=0.7382
  S

c:\Users\bgmanuel\anaconda3\envs\py310\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\bgmanuel\anaconda3\envs\py310\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


  Validation -> PSNR: 28.14, SSIM: 0.7929, G_loss: 22.2684


In [ ]:
results = model.evaluate(test_dataset)

metrics_dict = {
    "eval_loss": float(results["avg_g_loss"]),
    "eval_psnr": float(results["avg_psnr"]),
    "eval_ssim": float(results["avg_ssim"]),
    "final_train_loss": epoch_losses.get("g_loss", None)[-1] if epoch_losses.get("g_loss", None) else None,
    "final_val_loss": epoch_losses.get("val_g_loss", None)[-1] if epoch_losses.get("val_g_loss", None) else None,
    "final_train_psnr": epoch_losses.get("psnr", None)[-1] if epoch_losses.get("psnr", None) else None,
    "final_val_psnr": epoch_losses.get("val_psnr", None)[-1] if epoch_losses.get("val_psnr", None) else None,
    "final_train_ssim": epoch_losses.get("ssim", None)[-1] if epoch_losses.get("ssim", None) else None,
    "final_val_ssim": epoch_losses.get("val_ssim", None)[-1] if epoch_losses.get("val_ssim", None) else None,
    "epoch_time_sec": time_cb.mean_time_value(),
    "memory": mem_cb.as_dict()
}

Evaluating model on test dataset...
Evaluation Results:
  Average PSNR: 28.0879
  Average SSIM: 0.7919
  Average G Loss: 22.5064


In [9]:
# Save the trained model
model.save(directory=directory, timestamp=timestamp)

Generator model saved to models/ESRGAN/ESRGAN_20250909_193854\ESRGAN_generator_x2_20250909_193854.h5
Discriminator model saved to models/ESRGAN/ESRGAN_20250909_193854\ESRGAN_discriminator_x2_20250909_193854.h5


In [10]:
# Save evaluation/time/memory metrics next to the model
metrics_path = os.path.abspath(os.path.join(os.getcwd(), f"models/ESRGAN/ESRGAN_{timestamp}/ESRGAN_{timestamp}_metrics.pkl"))

with open(metrics_path, "wb") as f:
    pickle.dump(metrics_dict, f)
    
print(f"Saved metrics to {metrics_path}")

Saved metrics to c:\Users\bgmanuel\InteligenciaArtificial\MasterInteligenciaArtificial\Periodo2\TFM\Super-Resolution-Images-for-3D-Printing-Defect-Detection\SRModels\deep_learning_models\models\ESRGAN\ESRGAN_20250909_193854\ESRGAN_20250909_193854_metrics.pkl
